# Introduction
 In this assignment, you will do exercises on arcpy.Exists(), arcpy.Walk(), arcpy.da.Describe(), python list comprehension, arcpy.da.SearchCursor, and arcpy.da.InsertCursor(). In each section, a code block is provided but not ready to run. You need to add this notebook to ArcGIS Pro, modify each block to make it runable, keep the output message , and write an explanation of the code block. 

 Data preparation: 
 Use the same zip file from assignment 4 for this exercise. The zip file contains a geodatabase and a folder with shapefles. Download the data from the [data](../data) folder of the github website. Extract the zip file and use the geodatabase and the folder containing the shapefiles accordingly.  

## Read a text file for data and write them into a feature class using InsertCursor

- This code uses the points.csv file in the same folder of the notebook 
- Modify the geodatabase_path variable to make it work and write the "points" feature class in your geodatabase


In [ ]:
import arcpy
import csv

# Prompt for user input
csv_file_path = "Enter the path to the CSV file: "
geodatabase_path = "Enter the path to the geodatabase: "
feature_class_name = "points"

# Determine the number of fields and field names from the CSV header
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    header = next(csv_reader)
    num_fields = len(header)

# Create a new point feature class
sr = arcpy.SpatialReference(4326)  
arcpy.CreateFeatureclass_management(geodatabase_path, feature_class_name, "POINT", spatial_reference=sr)

# Add fields to the feature class based on the CSV header
field_info = arcpy.ListFields(geodatabase_path + "/" + feature_class_name)
field_names = [field.name for field in field_info]
for field in header:
    if field not in field_names:
        arcpy.AddField_management(geodatabase_path + "/" + feature_class_name, field, "DOUBLE")

# Create a cursor for inserting point features with additional fields
cursor_fields = ["SHAPE@X", "SHAPE@Y"] + header
with arcpy.da.InsertCursor(geodatabase_path + "/" + feature_class_name, cursor_fields) as cursor:
    # Read data from the CSV file and create point features with attributes
    with open(csv_file_path, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        
        # Skip the header row (if present)
        next(csv_reader, None)
        
        for row in csv_reader:
            x = float(row[0])
            y = float(row[1])
            additional_fields = row
            cursor.insertRow((x, y) + tuple(additional_fields))

print(f"New point feature class '{feature_class_name}' created in '{geodatabase_path}' with {num_fields} fields.")



**Edit this block to answer these three questions**

Q1: Line by line, explain what was done in the code. (20 pnts)

Q2: What does the cod 4326 represent? (5 pnts)

Q3: Explain what are SHAPE@X and SHAPE@Y (5 pnts)

## Use arcpy.Exist()

- This code checks the existence of a specified dataset within an ArcGIS workspace.
- Fill the dataset_name and workspace_path variables with the database and the feature class name (points) from the last block
- Print a message indicating whether the dataset exists or not.

In [ ]:
import arcpy
import os

# Prompt for user input
dataset_name = "points"
workspace_path = "Enter the workspace path: "
fullname = os.path.join(workspace_path,dataset_name)
# Check if the dataset exists
if arcpy.Exists(fullname):
    print(f"The dataset '{dataset_name}' exists in the workspace.")
else:
    print(f"The dataset '{dataset_name}' does not exist in the workspace.")

**Edit this block to answer the question**

Q4: Line by line describe what was done by the code in the block above. (10 pnts) 



## Use arcpy.Walk()

- This code uses arcpy.Walk() to iterate through all feature datasets in the geodatabase you just used.
- Modify the name of "workspace" to make the code work
- Run the code to list all the feature classes within each dataset.


In [ ]:
import arcpy

# Define the workspace
workspace = "C:/path_to_geodatabase.gdb" # note that you need to put a full path name for this workspace, even running it in ArcGIS Pro. 

# Use arcpy.Walk() to iterate through feature datasets
for dirpath, dirnames, filenames in arcpy.da.Walk(workspace, datatype="FeatureClass"):
    for filename in filenames:
        print(f"Feature Class in {dirpath}: {filename}")

**Edit this block to answer the question**

Q5: line by line, describe what was done by the code in the block above. (10 pnts)


## Use List Comprehension

- The following block uses list comprehension to generate a list of .shp files in a specified folder.
- Choose the folder name from assignment 4 data/paris subfolder where many shapefiles are located and use it for folder_path
- Run the code to print the list of file names


- In the second block, the code combines the os.walk() function with the list comprehension to  list all shapefiles in a folder including subfolders



In [ ]:
import os

# Specify the folder path
folder_path = "C:/path_to_folder"

# Use list comprehension to generate a list of .shp files
shp_files = [file for file in os.listdir(folder_path) if file.endswith(".shp")]

# Print the list of .shp files
print("Shapefiles in the folder:")
for shp_file in shp_files:
    print(shp_file)

In [ ]:
import os
# Specify the folder path
folder_path = "C:/path_to_folder"

print("Shapefiles in the folder:")
for root, dirs, files in os.walk(folder_path):
    shp_files = [file for file in files if file.endswith(".shp")]
    for shp_file in shp_files:
            print(shp_file)

**Edit this block to answer the question**

Q6: Describe what was done by the code in first blocks above. (10 pnts)

Q7: Describe what was done by the code in second blocks above. (10 pnts)

## Use arcpy.da.SearchCursor

- This code uses arcpy.da.SearchCursor to extract attribute information from a feature class.
- Enter the name of the "points" feature class and the field(s) from the feature class to extract.
- Run and display the extracted data.


In [ ]:
import arcpy

# Prompt for user input
fc_path = "Enter the feature class path: "
fields_to_extract = ["Enter the field(s) to extract (comma-separated): "]

# Use arcpy.da.SearchCursor to extract data
with arcpy.da.SearchCursor(fc_path, fields_to_extract) as cursor:
    print("Extracted Data:")
    for row in cursor:
        print([row[i] for i in range(len(fields_to_extract))])

**Edit this block to answer the question**

Q8: Line by line, explain what was done in the code. 10 pnts 

## Use addField and field Calculator

- Run the following code block (with "points" feature class added to the last map before you switched into the notebook interface)

In [ ]:
import arcpy
fc = "points"
newfieldName = "all"
arcpy.AddField_management(fc, newfieldName, "DOUBLE")
expression = "sum(!Field1!,!Field2!,!Field3!)"
codeblock = """
def sum(*fields):
    sum = 0
    for field in fields:
        sum += field
    return sum
"""
arcpy.CalculateField_management(fc, newfieldName, expression, "", codeblock)

**Edit this block to answer the questions**

Q9: line by line, describe what was done by the code in the block above. 10 pnts

Q10: Open the attribute table of "points" and check if the attribute table has a new filed "all" and correct values. Right-click the "all" field and click field calculator. Describe what you see in the field calculator interface. Compare the python code versus the field calculator interface. 10 pnts